In [ ]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2 as c
import numpy as np

face_classifier = c.CascadeClassifier('haarcascade_frontalface_default.xml')
classifier = load_model('Emotion_Detection.h5')

class_labels = ['Angry','Happy','Neutral','Sad','Surprised']

cap = c.VideoCapture(0)



while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    gray = c.cvtColor(frame,c.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        c.rectangle(frame,(x,y),(x+w,y+h),(0,0,0),10)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = c.resize(roi_gray,(48,48),interpolation=c.INTER_AREA)


        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)


            preds = classifier.predict(roi)[0]
            #print("\nprediction = ",preds)
            label=class_labels[preds.argmax()]
            #print("\nprediction max = ",preds.argmax())
            #print("\nlabel = ",label)
            label_position = (x,y)
            c.putText(frame,label,label_position,c.FONT_HERSHEY_SIMPLEX,2,(40, 224, 129),5)
        else:
            c.putText(frame,'Please make certain there is a face in front of the Camera.',(20,60),c.FONT_HERSHEY_SIMPLEX,2,(40, 224, 129),3)
    c.imshow('Emotion Detector',frame)
    if c.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
c.destroyAllWindows()
